In [1]:
#manipulação de dados
import pandas as pd
import numpy as np

#automação
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#web scraping
import requests
from bs4 import BeautifulSoup

#string e data
import unidecode

#tempo
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import re

# SEMANA

In [3]:
lista = ['seg','ter','qua','qui','sex']

lista_abas = []
for t in range(1, 3):
    for i in lista:
        #print(str(i)+'_1_'+str(t))
        lista_abas.append(str(i)+'_1_'+str(t))
        
#lista_abas

fd = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo']) #df vazio

count = 0

for h in lista_abas:
    if h[-1:] == '1':
        periodo = 'MANHÃ'
    elif h[-1:] == '2':
        periodo = 'TARDE'
        
    tempo = h
    df = pd.read_excel('Escala SEMANA.xlsx', index_col=periodo, sheet_name=tempo)

    for coluna in df.columns: #para cada coluna
        #print(coluna, '='*20)
        for ind in df.index: #para cada index
            #print(ind,'='*10)
            if pd.isnull(df.loc[ind, coluna]) == True: #verificar se == NA
                continue
            elif (df.loc[ind, coluna] == 'EXTRA') or (df.loc[ind, coluna] == 'Cobertura'):
                continue
            else: #capturar o nome, setor, unidade
                fd.loc[count] = [df.loc[ind, coluna].replace("*",""), ind, coluna, periodo, tempo]

                count += 1
        
    fd['Unidade'] = fd['Unidade'].apply(lambda x: unidecode.unidecode(x.upper()))

fd

,Nome,Setor,Unidade,Periodo,tempo
0,Natan,PRÉ,ANALIA,MANHÃ,seg_1_1
1,Vanessa,AB,ANALIA,MANHÃ,seg_1_1
2,Alex,CC1,ANALIA,MANHÃ,seg_1_1
3,Ana,CC2,ANALIA,MANHÃ,seg_1_1
4,Cadu,CC4,ANALIA,MANHÃ,seg_1_1
...,...,...,...,...,...
1313,Queiroz,CC12,ADM,TARDE,sex_1_2
1314,Joao,CC13,ADM,TARDE,sex_1_2
1315,Torrelio,CC14,ADM,TARDE,sex_1_2
1316,Icaro,CC15,ADM,TARDE,sex_1_2


In [4]:
print(fd['Setor'].unique())
print(fd['Unidade'].unique())
print(fd['tempo'].unique())

['PRÉ' 'AB' 'CC1' 'CC2' 'CC4' 'CC7' 'CC8' 'CC9' 'CC10' 'CC11' 'CC12'
 'CC13' 'CC14' 'CC15' 'CC16' 'CC17' 'CC19' '11-13: HA' 'CC3' 'CC5' 'CC6'
 'CD + CC' 'CC18' 'CC20' 'CC21' 'AMB']
['ANALIA' 'VILLA' 'AVICCENA' 'ITAIM' 'VNS' 'MATE' 'MORUMBI' 'JABA'
 'OSASCO' 'ALPHA' 'SANTA ISABEL' 'ADM']
['seg_1_1' 'ter_1_1' 'qua_1_1' 'qui_1_1' 'sex_1_1' 'seg_1_2' 'ter_1_2'
 'qua_1_2' 'qui_1_2' 'sex_1_2']


In [5]:
#CORRIGINDO HORARIOS
for i in range(0, len(fd)):
    if fd['Periodo'][i] == 'MANHÃ':
        if fd['Setor'][i][:2] == "CC":
            fd['Setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        else:
            continue
    elif fd['Periodo'][i] == 'TARDE':
        if fd['Setor'][i][:2] == "CC":
            fd['Setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        else:
            continue
    else:
        continue

In [6]:
fd['ENTRADA'] = ''
fd['SAIDA'] = ''
fd['DURAÇÃO'] = ''

#MANHÃ######################################
fd['Setor'][(fd['Setor'] == 'PRÉ') & (fd['Periodo'] == 'MANHÃ')] = 'SEMANA - MANHA - AV.PRÉ ANESTÉSICA'
fd['Setor'][(fd['Setor'] == 'AB') & (fd['Periodo'] == 'MANHÃ')] = 'SEMANA - MANHA - AMBULATÓRIO'

fd['ENTRADA'][fd['Periodo'] == 'MANHÃ'] = '07:00'
fd['SAIDA'][fd['Periodo'] == 'MANHÃ'] = '13:00'
fd['DURAÇÃO'][fd['Periodo'] == 'MANHÃ'] = '06:00'

fd['Setor'][(fd['Setor'] == 'CD + CC') & (fd['Periodo'] == 'MANHÃ')] = 'SEMANA - MANHA - C.DIAGNÓSTICO'

fd['ENTRADA'][fd['Setor'] == '11-13: HA'] = '11:00'
fd['SAIDA'][fd['Setor'] == '11-13: HA'] = '13:00'
fd['DURAÇÃO'][fd['Setor'] == '11-13: HA'] = '02:00'
fd['Setor'][fd['Setor'] == '11-13: HA'] = 'SEMANA - MANHA - C.CIRURGICO'

#TARDE######################################
fd['Setor'][(fd['Setor'] == 'PRÉ') & (fd['Periodo'] == 'TARDE')] = 'SEMANA - TARDE - AV.PRÉ ANESTÉSICA'
fd['Setor'][(fd['Setor'] == 'AMB') & (fd['Periodo'] == 'TARDE')] = 'SEMANA - TARDE - AMBULATÓRIO'

fd['ENTRADA'][fd['Periodo'] == 'TARDE'] = '13:00'
fd['SAIDA'][fd['Periodo'] == 'TARDE'] = '19:00'
fd['DURAÇÃO'][fd['Periodo'] == 'TARDE'] = '06:00'

#fd['ENTRADA'][fd['Setor'] == '17:00 - 23:00'] = '17:00'
#fd['SAIDA'][fd['Setor'] == '17:00 - 23:00'] = '23:00'
#fd['DURAÇÃO'][fd['Setor'] == '17:00 - 23:00'] = '06:00'

fd['Setor'][(fd['Setor'] == 'CD + CC') & (fd['Periodo'] == 'MANHA')] = 'SEMANA - MANHA - C.DIAGNOSTICO'
fd['Setor'][(fd['Setor'] == 'CD + CC') & (fd['Periodo'] == 'TARDE')] = 'SEMANA - TARDE - C.DIAGNOSTICO'

fd['Setor'].unique()

array(['SEMANA - MANHA - AV.PRÉ ANESTÉSICA',
       'SEMANA - MANHA - AMBULATÓRIO', 'SEMANA - MANHA - C.CIRURGICO',
       'SEMANA - MANHA - C.DIAGNÓSTICO',
       'SEMANA - TARDE - AV.PRÉ ANESTÉSICA',
       'SEMANA - TARDE - C.CIRURGICO', 'SEMANA - TARDE - AMBULATÓRIO',
       'SEMANA - TARDE - C.DIAGNOSTICO'], dtype=object)

In [7]:
#display(fd[fd['Setor'] == '11:00 - 17:00'])
#display(fd[fd['Setor'] == '17:00-23:00'])

In [8]:
#confirmando dados em tabela
#a = fd[fd['tempo'] == 'seg_1_1'].groupby(['Setor', 'Unidade']).size().unstack()
a = fd.groupby(['Setor', 'Periodo']).size().unstack()
for i in a.columns:
    a[i].fillna(0, inplace = True)
    a[i] = a[i].apply(lambda x: int(x))
    #a[i] = a[i].apply(lambda x: round((x/len(fd))*100, 2))
display(a)

Periodo,MANHÃ,TARDE
Setor,,
SEMANA - MANHA - AMBULATÓRIO,6,0
SEMANA - MANHA - AV.PRÉ ANESTÉSICA,29,0
SEMANA - MANHA - C.CIRURGICO,635,0
SEMANA - MANHA - C.DIAGNÓSTICO,5,0
SEMANA - TARDE - AMBULATÓRIO,0,3
SEMANA - TARDE - AV.PRÉ ANESTÉSICA,0,26
SEMANA - TARDE - C.CIRURGICO,0,611
SEMANA - TARDE - C.DIAGNOSTICO,0,3


In [9]:
print(fd['Setor'].unique())
print(fd['Unidade'].unique())
print(fd['tempo'].unique())

['SEMANA - MANHA - AV.PRÉ ANESTÉSICA' 'SEMANA - MANHA - AMBULATÓRIO'
 'SEMANA - MANHA - C.CIRURGICO' 'SEMANA - MANHA - C.DIAGNÓSTICO'
 'SEMANA - TARDE - AV.PRÉ ANESTÉSICA' 'SEMANA - TARDE - C.CIRURGICO'
 'SEMANA - TARDE - AMBULATÓRIO' 'SEMANA - TARDE - C.DIAGNOSTICO']
['ANALIA' 'VILLA' 'AVICCENA' 'ITAIM' 'VNS' 'MATE' 'MORUMBI' 'JABA'
 'OSASCO' 'ALPHA' 'SANTA ISABEL' 'ADM']
['seg_1_1' 'ter_1_1' 'qua_1_1' 'qui_1_1' 'sex_1_1' 'seg_1_2' 'ter_1_2'
 'qua_1_2' 'qui_1_2' 'sex_1_2']


In [10]:
fd['SEMANA'] = '1'

In [11]:
fd.isnull().sum()

Nome       0
Setor      0
Unidade    0
Periodo    0
tempo      0
ENTRADA    0
SAIDA      0
DURAÇÃO    0
SEMANA     0
dtype: int64

# SEMANA - NOTURNO

In [12]:
fd_no = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo']) #df vazio

count = 0

lista_abas = ['seg','ter','qua','qui','sex']
periodo = "NOITE"

for h in lista_abas:
    tempo = h
    df = pd.read_excel('Escala NOITE SEMANA.xlsx', index_col=periodo, sheet_name=tempo)

    for coluna in df.columns: #para cada coluna
        #print(coluna, '='*20)
        for ind in df.index: #para cada index
            #print(ind,'='*10)
            if pd.isnull(df.loc[ind, coluna]) == True: #verificar se == NA
                continue
            elif (df.loc[ind, coluna] == 'EXTRA') or (df.loc[ind, coluna] == 'Cobertura'):
                continue
            else: #capturar o nome, setor, unidade
                fd_no.loc[count] = [df.loc[ind, coluna].replace("*",""), ind, coluna, 'NOTURNO', tempo]

                count += 1
        
    fd_no['Unidade'] = fd_no['Unidade'].apply(lambda x: unidecode.unidecode(x.upper()))

fd_no

,Nome,Setor,Unidade,Periodo,tempo
0,Astur,19-23: HA,ANALIA,NOTURNO,seg
1,Paola,19-07: DT3,ANALIA,NOTURNO,seg
2,Maciel,19-07: PT,ANALIA,NOTURNO,seg
3,Nunes,19-07: PT2,ANALIA,NOTURNO,seg
4,R.Bueno,19-23: HA,VILLA,NOTURNO,seg
...,...,...,...,...,...
147,Nathalia,19-07: PT,OSASCO,NOTURNO,sex
148,Haga,19-07: PT2,OSASCO,NOTURNO,sex
149,Hedicleber,19-07: PT,ALPHA,NOTURNO,sex
150,Diogo,19-07: DT3,SANTA ISABEL,NOTURNO,sex


In [13]:
print(fd_no['Setor'].unique())
print(fd_no['Unidade'].unique())
print(fd_no['Periodo'].unique())

['19-23: HA' '19-07: DT3' '19-07: PT' '19-07: PT2' '19-07: DT2'
 '19-07: DT']
['ANALIA' 'VILLA' 'AVICCENA' 'ITAIM' 'VNS' 'MATE' 'MORUMBI' 'JABA'
 'OSASCO' 'ALPHA' 'SANTA ISABEL']
['NOTURNO']


In [14]:
fd_no['ENTRADA'] = ''
fd_no['SAIDA'] = ''
fd_no['DURAÇÃO'] = ''

for i in range(0, len(fd_no)):
    if fd_no['Setor'][i] == '19-23: HA':
        fd_no['ENTRADA'][i] = '19:00'
        fd_no['SAIDA'][i] = '23:00'
        fd_no['DURAÇÃO'][i] = '04:00'
        fd_no['Setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        
    elif (fd_no['Setor'][i] == '19-07: DT') or (fd_no['Setor'][i] == '19-07: DT2') or (fd_no['Setor'][i] == '19-07: DT3'):
        fd_no['ENTRADA'][i] = '19:00'
        fd_no['SAIDA'][i] = '07:00'
        fd_no['DURAÇÃO'][i] = '12:00'
        fd_no['Setor'][i] = 'SEMANA - NOTURNO - DISTANCIA'
        
    elif (fd_no['Setor'][i] == '19-07: PT') or (fd_no['Setor'][i] == '19-07: PT2'):
        fd_no['ENTRADA'][i] = '19:00'
        fd_no['SAIDA'][i] = '07:00'
        fd_no['DURAÇÃO'][i] = '12:00'
        fd_no['Setor'][i] = 'SEMANA - NOTURNO - PLANTAO'
        
fd_no

,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO
0,Astur,SEMANA - NOTURNO - SAIDA,ANALIA,NOTURNO,seg,19:00,23:00,04:00
1,Paola,SEMANA - NOTURNO - DISTANCIA,ANALIA,NOTURNO,seg,19:00,07:00,12:00
2,Maciel,SEMANA - NOTURNO - PLANTAO,ANALIA,NOTURNO,seg,19:00,07:00,12:00
3,Nunes,SEMANA - NOTURNO - PLANTAO,ANALIA,NOTURNO,seg,19:00,07:00,12:00
4,R.Bueno,SEMANA - NOTURNO - SAIDA,VILLA,NOTURNO,seg,19:00,23:00,04:00
...,...,...,...,...,...,...,...,...
147,Nathalia,SEMANA - NOTURNO - PLANTAO,OSASCO,NOTURNO,sex,19:00,07:00,12:00
148,Haga,SEMANA - NOTURNO - PLANTAO,OSASCO,NOTURNO,sex,19:00,07:00,12:00
149,Hedicleber,SEMANA - NOTURNO - PLANTAO,ALPHA,NOTURNO,sex,19:00,07:00,12:00
150,Diogo,SEMANA - NOTURNO - DISTANCIA,SANTA ISABEL,NOTURNO,sex,19:00,07:00,12:00


# FIM DE SEMANA

In [15]:
lista_abas = []
for i in range(1,3):
    for t in range(1,5):
        lista_abas.append('sab_'+str(t)+'_'+str(i))
        
lista_abas

['sab_1_1',
 'sab_2_1',
 'sab_3_1',
 'sab_4_1',
 'sab_1_2',
 'sab_2_2',
 'sab_3_2',
 'sab_4_2']

In [16]:
fds_sab = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo']) #df vazio

count = 0

for h in lista_abas:
    if h[-1:] == '1':
        periodo = 'DIA'
    elif h[-1:] == '2':
        periodo = 'NOITE'
        
    tempo = h
    df = pd.read_excel('Escala FDS.xlsx', index_col=periodo, sheet_name=tempo)

    for coluna in df.columns: #para cada coluna
        #print(coluna, '='*20)
        for ind in df.index: #para cada index
            #print(ind,'='*10)
            if pd.isnull(df.loc[ind, coluna]) == True: #verificar se == NA
                continue
            elif (df.loc[ind, coluna] == 'EXTRA') or (df.loc[ind, coluna] == 'Cobertura'):
                continue
            else: #capturar o nome, setor, unidade
                fds_sab.loc[count] = [df.loc[ind, coluna].replace("*",""), ind, coluna, periodo, tempo]

                count += 1
        
    fds_sab['Unidade'] = fds_sab['Unidade'].apply(lambda x: unidecode.unidecode(x.upper()))
    
fds_sab

,Nome,Setor,Unidade,Periodo,tempo
0,Helena,PRÉ,ANALIA,DIA,sab_1_1
1,Aline,CC1,ANALIA,DIA,sab_1_1
2,Cortes,CC2,ANALIA,DIA,sab_1_1
3,Francisco,CC4,ANALIA,DIA,sab_1_1
4,Giovanelli,CC5,ANALIA,DIA,sab_1_1
...,...,...,...,...,...
503,Leticia,DT,OSASCO,NOITE,sab_4_2
504,Pereira,PT,OSASCO,NOITE,sab_4_2
505,Stefany,PT,ALPHA,NOITE,sab_4_2
506,Elaine,DT,SANTA ISABEL,NOITE,sab_4_2


In [17]:
fds_sab[fds_sab['Nome'] == 'Helena']

,Nome,Setor,Unidade,Periodo,tempo
0,Helena,PRÉ,ANALIA,DIA,sab_1_1
295,Helena,CC4,OSASCO,DIA,sab_3_1


In [18]:
#CORRIGINDO HORARIOS
for i in range(0, len(fds_sab)):
    if fds_sab['Periodo'][i] == 'DIA':
        if fds_sab['Setor'][i][:2] == "CC":
            fds_sab['Setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        else:
            continue
    elif fds_sab['Periodo'][i] == 'NOITE':
        if fds_sab['Setor'][i][:2] == "CC":
            fds_sab['Setor'][i] = 'FDS - PLANTAO'
        else:
            continue
    else:
        continue
#obs: ainda esta sem horario

In [19]:
fds_sab.head()

,Nome,Setor,Unidade,Periodo,tempo
0,Helena,PRÉ,ANALIA,DIA,sab_1_1
1,Aline,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1
2,Cortes,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1
3,Francisco,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1
4,Giovanelli,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1


In [20]:
fds_sab['ENTRADA'] = ''
fds_sab['SAIDA'] = ''
fds_sab['DURAÇÃO'] = ''

fds_sab['ENTRADA'][fds_sab['Setor'] == 'FDS - C.CIRURGICO ROTINA'] = "07:00"
fds_sab['SAIDA'][fds_sab['Setor'] == 'FDS - C.CIRURGICO ROTINA'] = '13:00'
fds_sab['DURAÇÃO'][fds_sab['Setor'] == 'FDS - C.CIRURGICO ROTINA'] = '06:00'

In [21]:
#DIA######################################
fds_sab['Setor'][(fds_sab['Setor'] == 'PRÉ') & (fds_sab['Periodo'] == 'DIA')] = 'FDS - MANHA - AV.PRÉ ANESTÉSICA'

fds_sab['ENTRADA'][fds_sab['Periodo'] == 'DIA'] = '07:00'
fds_sab['SAIDA'][fds_sab['Periodo'] == 'DIA'] = '13:00'
fds_sab['DURAÇÃO'][fds_sab['Periodo'] == 'DIA'] = '06:00'

In [22]:
dif_hor = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo','ENTRADA','SAIDA','DURAÇÃO']) #df vazio

count = 0
for i in fds_sab[fds_sab['Setor'] == 'DT'].index.tolist():
    dif_hor.loc[count] = [fds_sab['Nome'][i], 'FDS - D1', fds_sab['Unidade'][i], 'TARDE', fds_sab['tempo'][i],
                          '13:00','19:00','06:00']
    
    dif_hor.loc[count+1] = [fds_sab['Nome'][i], 'FDS - D1', fds_sab['Unidade'][i], 'NOTURNO', fds_sab['tempo'][i],
                            '19:00','07:00','12:00']
    
    count += 2

dif_hor    
fds_sab.drop(fds_sab[fds_sab['Setor'] == 'DT'].index.tolist(), inplace = True)

fds_sab = pd.concat([fds_sab, dif_hor])
fds_sab.reset_index(drop=True, inplace=True)

In [23]:
dif_hor = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo','ENTRADA','SAIDA','DURAÇÃO']) #df vazio

count = 0
for i in fds_sab[fds_sab['Setor'] == 'PT'].index.tolist():
    dif_hor.loc[count] = [fds_sab['Nome'][i], 'FDS - PLANTAO', fds_sab['Unidade'][i], 'TARDE', fds_sab['tempo'][i],
                          '13:00','19:00','06:00']
    
    dif_hor.loc[count+1] = [fds_sab['Nome'][i], 'FDS - PLANTAO', fds_sab['Unidade'][i], 'NOTURNO', fds_sab['tempo'][i],
                            '19:00','07:00','12:00']
    
    count += 2

dif_hor    
fds_sab.drop(fds_sab[fds_sab['Setor'] == 'PT'].index.tolist(), inplace = True)

fds_sab = pd.concat([fds_sab, dif_hor])
fds_sab.reset_index(drop=True, inplace=True)

In [24]:
dif_hor = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo','ENTRADA','SAIDA','DURAÇÃO']) #df vazio

count = 0
for i in fds_sab[fds_sab['Setor'] == 'PT2'].index.tolist():
    dif_hor.loc[count] = [fds_sab['Nome'][i], 'FDS - PLANTAO', fds_sab['Unidade'][i], 'TARDE', fds_sab['tempo'][i],
                          '13:00','19:00','06:00']
    
    dif_hor.loc[count+1] = [fds_sab['Nome'][i], 'FDS - PLANTAO', fds_sab['Unidade'][i], 'NOTURNO', fds_sab['tempo'][i],
                            '19:00','07:00','12:00']
    
    count += 2

dif_hor    
fds_sab.drop(fds_sab[fds_sab['Setor'] == 'PT2'].index.tolist(), inplace = True)

fds_sab = pd.concat([fds_sab, dif_hor])
fds_sab.reset_index(drop=True, inplace=True)

In [25]:
print(fds_sab['Setor'].unique())
print(fds_sab['Unidade'].unique())
print(fds_sab['tempo'].unique())

['FDS - MANHA - AV.PRÉ ANESTÉSICA' 'FDS - C.CIRURGICO ROTINA' 'FDS - D1'
 'FDS - PLANTAO']
['ANALIA' 'VILLA' 'AVICCENA' 'ITAIM' 'STAR' 'MATE' 'MORUMBI' 'JABAQUARA'
 'OSASCO' 'ALPHA' 'SANTA ISABEL']
['sab_1_1' 'sab_2_1' 'sab_3_1' 'sab_4_1' 'sab_1_2' 'sab_2_2' 'sab_3_2'
 'sab_4_2']


In [26]:
fds_sab.sample(50)

,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO
401,Buto,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_4_1,07:00,13:00,06:00
292,Assuncao,FDS - C.CIRURGICO ROTINA,OSASCO,DIA,sab_3_1,07:00,13:00,06:00
244,Dardengo,FDS - C.CIRURGICO ROTINA,ITAIM,DIA,sab_3_1,07:00,13:00,06:00
321,Giovanelli,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_4_1,07:00,13:00,06:00
429,P.Neves,FDS - D1,ANALIA,TARDE,sab_2_2,13:00,19:00,06:00
162,Astur,FDS - MANHA - AV.PRÉ ANESTÉSICA,MORUMBI,DIA,sab_2_1,07:00,13:00,06:00
388,Emerson,FDS - C.CIRURGICO ROTINA,JABAQUARA,DIA,sab_4_1,07:00,13:00,06:00
523,Luiz,FDS - PLANTAO,MORUMBI,TARDE,sab_2_2,13:00,19:00,06:00
70,Wong,FDS - C.CIRURGICO ROTINA,MORUMBI,DIA,sab_1_1,07:00,13:00,06:00
144,Kaie,FDS - C.CIRURGICO ROTINA,ITAIM,DIA,sab_2_1,07:00,13:00,06:00


In [27]:
print(fds_sab['Setor'].unique())
print(fds_sab['Unidade'].unique())
print(fds_sab['tempo'].unique())

['FDS - MANHA - AV.PRÉ ANESTÉSICA' 'FDS - C.CIRURGICO ROTINA' 'FDS - D1'
 'FDS - PLANTAO']
['ANALIA' 'VILLA' 'AVICCENA' 'ITAIM' 'STAR' 'MATE' 'MORUMBI' 'JABAQUARA'
 'OSASCO' 'ALPHA' 'SANTA ISABEL']
['sab_1_1' 'sab_2_1' 'sab_3_1' 'sab_4_1' 'sab_1_2' 'sab_2_2' 'sab_3_2'
 'sab_4_2']


In [28]:
a = fds_sab.groupby(['Setor', 'Periodo']).size().unstack()
for i in a.columns:
    a[i].fillna(0, inplace = True)
    a[i] = a[i].apply(lambda x: int(x))
display(a)

Periodo,DIA,NOTURNO,TARDE
Setor,,,
FDS - C.CIRURGICO ROTINA,385,0,0
FDS - D1,0,40,40
FDS - MANHA - AV.PRÉ ANESTÉSICA,24,0,0
FDS - PLANTAO,0,59,59


# Concatenando dataframes

In [29]:
fds_sab

,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO
0,Helena,FDS - MANHA - AV.PRÉ ANESTÉSICA,ANALIA,DIA,sab_1_1,07:00,13:00,06:00
1,Aline,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1,07:00,13:00,06:00
2,Cortes,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1,07:00,13:00,06:00
3,Francisco,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1,07:00,13:00,06:00
4,Giovanelli,FDS - C.CIRURGICO ROTINA,ANALIA,DIA,sab_1_1,07:00,13:00,06:00
...,...,...,...,...,...,...,...,...
602,Max,FDS - PLANTAO,ANALIA,NOTURNO,sab_4_2,19:00,07:00,12:00
603,Cortes,FDS - PLANTAO,ITAIM,TARDE,sab_4_2,13:00,19:00,06:00
604,Cortes,FDS - PLANTAO,ITAIM,NOTURNO,sab_4_2,19:00,07:00,12:00
605,Thais,FDS - PLANTAO,MATE,TARDE,sab_4_2,13:00,19:00,06:00


In [30]:
fds_sab['SEMANA'] = ''

for i in range(0, len(fds_sab)):
    fds_sab['SEMANA'][i] = fds_sab['tempo'][i][4]

In [31]:
a = fds_sab.groupby(['tempo', 'SEMANA']).size().unstack()
for i in a.columns:
    a[i].fillna(" ", inplace = True)
display(a)

SEMANA,1,2,3,4
tempo,,,,
sab_1_1,107.0,,,
sab_1_2,50.0,,,
sab_2_1,,104.0,,
sab_2_2,,50.0,,
sab_3_1,,,101.0,
sab_3_2,,,50.0,
sab_4_1,,,,97.0
sab_4_2,,,,48.0


In [32]:
fd = pd.concat([fd, fds_sab])

In [33]:
#Há valores ausentes ?
fd[fd.isna()].count()

Nome       0
Setor      0
Unidade    0
Periodo    0
tempo      0
ENTRADA    0
SAIDA      0
DURAÇÃO    0
SEMANA     0
dtype: int64

# QUESTÃO DOS NOMES

In [34]:
sheet_id = 'https://docs.google.com/spreadsheets/d/1X2X7df9HCrwjA7phyyr7nzNU2pbqc_kDhBDe3FomVr4'
sheet_id = sheet_id+"/export?format=csv"
nome = pd.read_csv(sheet_id)

fd['controle'] = 'ok'
fd.reset_index(drop=True, inplace=True)


for i in range(0, len(fd)):
    nomezin = fd['Nome'][i].upper()
    #print(nomezin)
    if len(nome.loc[nome['NOME'] == nomezin]) == 0:
        fd['controle'][i] = 'erro'
        #continue
        #print("erro no nome:", nomezin, '='*20)
        #print('\n')
    else:
        ind = nome.loc[nome['NOME'] == nomezin].index.values[0]
        loc = nome['ANESTESISTA'][ind]
        fd['Nome'][i] = loc
        #print(loc)
        #print('\n')
        
fd['controle'].value_counts()

ok      1910
erro      15
Name: controle, dtype: int64

In [35]:
fd['Nome'][fd['controle'] == 'erro'].unique()

array(['Bellini', 'Saboia'], dtype=object)

In [36]:
fd['Nome'][fd['Nome'] == 'Cavichini'] = 'KAREN CAVICHINI'
fd['Nome'][fd['Nome'] == 'Machado'] = 'LUCAS ARDERBAL'
fd['Nome'][fd['Nome'] == 'Garcia'] = 'ISABELA GARCIA'

In [37]:
fd['Unidade'].unique()

array(['ANALIA', 'VILLA', 'AVICCENA', 'ITAIM', 'VNS', 'MATE', 'MORUMBI',
       'JABA', 'OSASCO', 'ALPHA', 'SANTA ISABEL', 'ADM', 'STAR',
       'JABAQUARA'], dtype=object)

In [38]:
fd['Unidade'][fd['Unidade'] == 'MATE'] = 'MATERNIDADE'
fd['Unidade'][fd['Unidade'] == 'JABA'] = 'JABAQUARA'
fd['Unidade'][fd['Unidade'] == 'ADM'] = 'ADMINISTRACAO'

# Verificação

In [39]:
#confirmando dados da tabela
#a = fd[fd['tempo'] == 'seg_1_1'].groupby(['Setor', 'Unidade']).size().unstack()
a = fd.groupby(['Setor', 'Unidade']).size().unstack()
for i in a.columns:
    a[i].fillna(0, inplace = True)
    a[i] = a[i].apply(lambda x: int(x))
    #a[i] = a[i].apply(lambda x: round((x/len(fd))*100, 2))
display(a)

Unidade,ADMINISTRACAO,ALPHA,ANALIA,AVICCENA,ITAIM,JABAQUARA,MATERNIDADE,MORUMBI,OSASCO,SANTA ISABEL,STAR,VILLA,VNS
Setor,,,,,,,,,,,,,
FDS - C.CIRURGICO ROTINA,0,8,52,12,63,51,21,47,44,23,20,44,0
FDS - D1,0,0,8,8,8,8,8,8,8,8,8,8,0
FDS - MANHA - AV.PRÉ ANESTÉSICA,0,4,3,0,4,3,0,3,4,0,0,3,0
FDS - PLANTAO,0,8,16,8,16,8,16,8,14,8,8,8,0
SEMANA - MANHA - AMBULATÓRIO,3,0,1,0,0,0,0,1,0,0,0,1,0
SEMANA - MANHA - AV.PRÉ ANESTÉSICA,1,0,5,0,4,5,0,4,5,0,0,5,0
SEMANA - MANHA - C.CIRURGICO,60,15,91,20,94,83,22,59,54,35,0,66,36
SEMANA - MANHA - C.DIAGNÓSTICO,0,0,0,0,0,0,0,5,0,0,0,0,0
SEMANA - TARDE - AMBULATÓRIO,3,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
a = fd.groupby(['Setor', 'Periodo']).size().unstack()
for i in a.columns:
    a[i].fillna(0, inplace = True)
    a[i] = a[i].apply(lambda x: int(x))
display(a)

Periodo,DIA,MANHÃ,NOTURNO,TARDE
Setor,,,,
FDS - C.CIRURGICO ROTINA,385,0,0,0
FDS - D1,0,0,40,40
FDS - MANHA - AV.PRÉ ANESTÉSICA,24,0,0,0
FDS - PLANTAO,0,0,59,59
SEMANA - MANHA - AMBULATÓRIO,0,6,0,0
SEMANA - MANHA - AV.PRÉ ANESTÉSICA,0,29,0,0
SEMANA - MANHA - C.CIRURGICO,0,635,0,0
SEMANA - MANHA - C.DIAGNÓSTICO,0,5,0,0
SEMANA - TARDE - AMBULATÓRIO,0,0,0,3


In [41]:
a = fd.groupby(['Unidade'])['Unidade'].count().reset_index(name='Freq')
a['per'] = a['Freq']*100/sum(a['Freq'])
display(a.sort_values(by = 'Unidade', ascending = True))

,Unidade,Freq,per
0,ADMINISTRACAO,123,6.389610
1,ALPHA,50,2.597403
2,ANALIA,265,13.766234
3,AVICCENA,68,3.532468
4,ITAIM,278,14.441558
5,JABAQUARA,243,12.623377
6,MATERNIDADE,89,4.623377
7,MORUMBI,204,10.597403
8,OSASCO,187,9.714286
9,SANTA ISABEL,108,5.610390


# Arrumando dias

In [34]:
dia = ['Segunda','Terça','Quarta','Quinta','Sexta','Sabado','Domingo']
cod = [2,3,4,5,6,7,8]

tabela_dia = pd.DataFrame({'dia_semana': dia, 'cod_dia_semana': cod})
tabela_dia

,dia_semana,cod_dia_semana
0,Segunda,2
1,Terça,3
2,Quarta,4
3,Quinta,5
4,Sexta,6
5,Sabado,7
6,Domingo,8


In [43]:
fd['tempo'].unique()

array(['seg_1_1', 'ter_1_1', 'qua_1_1', 'qui_1_1', 'sex_1_1', 'seg_1_2',
       'ter_1_2', 'qua_1_2', 'qui_1_2', 'sex_1_2', 'sab_1_1', 'sab_2_1',
       'sab_3_1', 'sab_4_1', 'sab_1_2', 'sab_2_2', 'sab_3_2', 'sab_4_2'],
      dtype=object)

In [44]:
fd['dia'] = ''

fd['dia'][fd['tempo']=='seg_1_1'] = 'Segunda'
fd['dia'][fd['tempo']=='seg_1_2'] = 'Segunda'

fd['dia'][fd['tempo']=='ter_1_1'] = 'Terça'
fd['dia'][fd['tempo']=='ter_1_2'] = 'Terça'

fd['dia'][fd['tempo']=='qua_1_1'] = 'Quarta'
fd['dia'][fd['tempo']=='qua_1_2'] = 'Quarta'

fd['dia'][fd['tempo']=='qui_1_1'] = 'Quinta'
fd['dia'][fd['tempo']=='qui_1_2'] = 'Quinta'

fd['dia'][fd['tempo']=='sex_1_1'] = 'Sexta'
fd['dia'][fd['tempo']=='sex_1_2'] = 'Sexta'

fd['dia'][fd['tempo']=='sab_1_1'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_2_1'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_3_1'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_4_1'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_1_2'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_2_2'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_3_2'] = 'Sabado'
fd['dia'][fd['tempo']=='sab_4_2'] = 'Sabado'

In [45]:
fd['semana'] = ''

for i in range(0, len(fd)):
    dia_semana = fd['dia'][i]
    if len(tabela_dia.loc[tabela_dia['dia_semana'] == dia_semana]) >= 1:
        ind = tabela_dia.loc[tabela_dia['dia_semana'] == dia_semana].index.values[0]
        loc = tabela_dia['cod_dia_semana'][ind]
        DIA_DA_SEMANA = loc
        fd['semana'][i] = DIA_DA_SEMANA
    else:
        continue
    
fd

,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO,SEMANA,controle,dia,semana
0,NATAN SOARES SILVEIRA,SEMANA - MANHA - AV.PRÉ ANESTÉSICA,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
1,VANESSA PACIULLO MAROSSI,SEMANA - MANHA - AMBULATÓRIO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
2,ALEXSANDER TAKASHI HAMADA,SEMANA - MANHA - C.CIRURGICO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
3,ANA JUNG DOCKO,SEMANA - MANHA - C.CIRURGICO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
4,CARLOS EDUARDO CORRÊA GRILO,SEMANA - MANHA - C.CIRURGICO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1920,MAXVILANO LIMA DE MACEDO,FDS - PLANTAO,ANALIA,NOTURNO,sab_4_2,19:00,07:00,12:00,4,ok,Sabado,7
1921,LEANDRO OLIVEIRA MARTINS CÔRTES,FDS - PLANTAO,ITAIM,TARDE,sab_4_2,13:00,19:00,06:00,4,ok,Sabado,7
1922,LEANDRO OLIVEIRA MARTINS CÔRTES,FDS - PLANTAO,ITAIM,NOTURNO,sab_4_2,19:00,07:00,12:00,4,ok,Sabado,7
1923,THAIS CHAVES PINHEIRO,FDS - PLANTAO,MATERNIDADE,TARDE,sab_4_2,13:00,19:00,06:00,4,ok,Sabado,7


In [46]:
a = fd.groupby(['tempo', 'semana']).size().unstack()
for i in a.columns:
    a[i].fillna(" ", inplace = True)
display(a)

semana,2,3,4,5,6,7
tempo,,,,,,
qua_1_1,,,139.0,,,
qua_1_2,,,133.0,,,
qui_1_1,,,,131.0,,
qui_1_2,,,,125.0,,
sab_1_1,,,,,,107.0
sab_1_2,,,,,,50.0
sab_2_1,,,,,,104.0
sab_2_2,,,,,,50.0
sab_3_1,,,,,,101.0


In [47]:
a = fd.groupby(['tempo', 'SEMANA']).size().unstack()
for i in a.columns:
    a[i].fillna(" ", inplace = True)
display(a)

SEMANA,1,2,3,4
tempo,,,,
qua_1_1,139.0,,,
qua_1_2,133.0,,,
qui_1_1,131.0,,,
qui_1_2,125.0,,,
sab_1_1,107.0,,,
sab_1_2,50.0,,,
sab_2_1,,104.0,,
sab_2_2,,50.0,,
sab_3_1,,,101.0,


# Execução

In [3]:
browser = webdriver.Firefox()

In [4]:
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_UserName"]').send_keys('gabriel_s_anjos@yahoo.com') #e-mail
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_Password"]').send_keys('Gabriel2022!') #senha
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_btnLogin"]').click() #login

In [5]:
browser.get('https://www.pegaplantao.com.br/EscalaModelo')

## Capturando setores

In [6]:
fd = pd.read_excel('continue.xlsx')
fd

,Unnamed: 0,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO,SEMANA,controle,dia,semana
0,0,HEDICLEBER PESTANA SEVERINO,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
1,1,KAIÊ FERNANDES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
2,2,LUCCAS VARELA GODOY,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
3,3,EDUARDO DALA ROSA SOARES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
4,4,PALOMA KIYOMI TAGUCHI LEMOS,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,1797,FERNANDO RIBEIRO SÁ,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6
1798,1798,GABRIELLA GADELHA SARMENTO,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6
1799,1799,MAIELY MARCOLIN,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6
1800,1800,PEDRO HENRIQUE MILHOMENS DA MOTA,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6


In [7]:
browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[1]/div/div[1]/div/div[2]/i[1]').click()

setor_html = browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[1]/div/div[1]/div/div[3]/div/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

In [8]:
lista_unidades = []
for i in fd['Unidade'].unique():
    lista_unidades.append(i)

lista_unidades.append('ADMINISTRACAO')
lista_unidades.append('HOSPITAL GABRIEL DO ANJOS')

print(lista_unidades)

['ALPHA', 'ANALIA', 'AVICCENA', 'ITAIM', 'JABAQUARA', 'MATERNIDADE', 'MORUMBI', 'OSASCO', 'SANTA ISABEL', 'VILLA', 'VNS', 'ADMINISTRACAO', 'HOSPITAL GABRIEL DO ANJOS']


In [9]:
scrap_unidades = pd.DataFrame(columns=['unidade','momento','plt']) #df vazio

count = 0
count_scraping = 1

for i in soup.find_all(class_="volatile-select-label"):
    #print(i.get_text().strip())
    unidadezinha_setor = unidecode.unidecode(i.get_text().strip().upper().replace('  ',' '))
    
    if unidadezinha_setor in lista_unidades:
        unidade_momento = unidadezinha_setor
        #print("unidade:", unidadezinha_setor)
    else:
        scrap_unidades.loc[count] = [unidade_momento, unidadezinha_setor, count_scraping]
        count += 1
        #print("setor:", unidadezinha_setor)
        
    count_scraping += 1
    
scrap_unidades

,unidade,momento,plt
0,ADMINISTRACAO,ADMINISTRATIVO,2
1,ADMINISTRACAO,MES ANTERIOR,3
2,ADMINISTRACAO,PERIODO DE DESCANSO - 1,4
3,ADMINISTRACAO,PERIODO DE DESCANSO - 2,5
4,ADMINISTRACAO,PERIODO DE DESCANSO - 3,6
...,...,...,...
194,VNS,SEMANA - TARDE - C.CIRURGICO,208
195,VNS,SEMANA - TARDE - COBERTURA EXTRA,209
196,VNS,SEMANA - NOTURNO - DISTANCIA,210
197,VNS,SEMANA - NOTURNO - PLANTAO,211


In [56]:
#browser.quit()

# Funções de Aplicações

In [10]:
def mudar_unidade_setor(count_control):
    unidade = df['Unidade'][count_control]
    setor = df['Setor'][count_control]
    
    #abrindo aba
    browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[1]/div/div[1]/div/div[2]/i[1]').click()
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))

    ind = scrap_unidades.loc[(scrap_unidades['unidade'] == unidade) & (scrap_unidades['momento'] == setor)].index.values[0]
    loc = scrap_unidades['plt'][ind]

    #selecionando setores
    browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li['+str(loc)+']').click()
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
    
#mudar_unidade_setor(0)

In [39]:
def criar_registro(count_control):
    dia_semana = df['dia'][count_control]
    ind = tabela_dia.loc[tabela_dia['dia_semana'] == dia_semana].index.values[0]
    loc = tabela_dia['cod_dia_semana'][ind]
    DIA_DA_SEMANA = loc
    
    SEMANA = df['SEMANA'][count_control]
    
    #click no criar plantão
    browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[3]/div[1]/table/tbody/tr['+str(SEMANA)+']/td['+str(DIA_DA_SEMANA)+']/button').click()
    
    #listar todos profissionais #AS VEZES DA PROBLEMA [5, 6]
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '/html/body/div[5]/div[2]/div/div[1]/div/div[3]/div/div/a')))
    browser.find_element("xpath", '/html/body/div[5]/div[2]/div/div[1]/div/div[4]/div[2]/div/a/span[2]/b').click()
    
    sleep(1.5)
    #inserindo nome #AS VEZES DA PROBLEMA [8,9]
    browser.find_element("xpath", '/html/body/div[8]/div/input').send_keys(df['Nome'][count_control])
    browser.find_element("xpath", '/html/body/div[8]/div/input').send_keys(Keys.RETURN)
    
    #inicio
    browser.find_element("xpath", '/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/input').click()
    browser.find_element("xpath", '/html/body/div[5]/div[2]/div/div[1]/div/div[2]/div/input').send_keys(df['ENTRADA'][count_control])
    
    #duração
    browser.find_element("xpath", '/html/body/div[5]/div[2]/div/div[1]/div/div[1]/div/input').click()
    browser.find_element("xpath", '/html/body/div[5]/div[2]/div/div[1]/div/div[1]/div/input').send_keys(df['DURAÇÃO'][count_control])
    
    #adicionar
    browser.find_element("xpath", '/html/body/div[5]/div[3]/button[1]').click()
    
criar_registro(i)
df['controle'][i] = "OK"
H += 1

# Ultimos Acertos

In [13]:
fd["Unidade"][fd["Unidade"] == "STAR"] = 'VNS'

In [14]:
fd.drop(fd[fd.Unidade == 'ADMINISTRACAO'].index, inplace=True)
fd.reset_index(drop=True, inplace=True)

In [15]:
fd['Unidade'] = fd['Unidade'].apply(lambda x: unidecode.unidecode(x.upper()))
fd['Setor'] = fd['Setor'].apply(lambda x: unidecode.unidecode(x.upper()))

In [16]:
count = 0
for unidade in fd['Unidade'].unique():
    selecao = fd[fd['Unidade'] == unidade]
    #print(unidade)
    for setor in selecao['Setor'].unique():
        if len(scrap_unidades.loc[(scrap_unidades['unidade'] == unidade) & (scrap_unidades['momento'] == setor)]) >= 1:
            count += 1
            #print("==>", setor)
        else:
            #print("erro ==>", setor)
            print('fd["Setor"][(fd["Unidade"] == "'+unidade+'") & (fd["Setor"] == "'+setor+'")] = ""')
print(count)

77


In [17]:
fd["Setor"][(fd["Unidade"] == "ANALIA") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - AVALIACAO PRE ROTINA"
fd["Setor"][(fd["Unidade"] == "VILLA") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - AVALIACAO PRE ROTINA"
fd["Setor"][(fd["Unidade"] == "ITAIM") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - AVALIACAO PRE ROTINA"
fd["Setor"][(fd["Unidade"] == "MORUMBI") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - AVALIACAO PRE ROTINA"
fd["Setor"][(fd["Unidade"] == "JABAQUARA") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - AVALIACAO PRE ROTINA"
fd["Setor"][(fd["Unidade"] == "OSASCO") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - AVALIACAO PRE ROTINA"
fd["Setor"][(fd["Unidade"] == "ALPHA") & (fd["Setor"] == "FDS - MANHA - AV.PRE ANESTESICA")] = "FDS - C.CIRURGICO ROTINA"
fd["Setor"][(fd["Unidade"] == "SANTA ISABEL") & (fd["Setor"] == "FDS - C.CIRURGICO ROTINA")] = "FDS - MANHA - C.CIRURGICO"

In [18]:
count = 0
for unidade in fd['Unidade'].unique():
    selecao = fd[fd['Unidade'] == unidade]
    #print(unidade)
    for setor in selecao['Setor'].unique():
        if len(scrap_unidades.loc[(scrap_unidades['unidade'] == unidade) & (scrap_unidades['momento'] == setor)]) >= 1:
            count += 1
            #print("==>", setor)
        else:
            #print("erro ==>", setor)
            print('fd["Setor"][(fd["Unidade"] == "'+unidade+'") & (fd["Setor"] == "'+setor+'")] = ""')
print(count)

77


# Teste Execução

In [23]:
df = fd.sort_values(['Unidade', 'Setor', 'semana', 'Periodo', 'SEMANA', 'Nome'])
df.reset_index(drop=True, inplace=True)
df_backup = df
df_backup.reset_index(drop=True, inplace=True)
df

,Unnamed: 0,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO,SEMANA,controle,dia,semana
0,0,HEDICLEBER PESTANA SEVERINO,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
1,1,KAIÊ FERNANDES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
2,2,LUCCAS VARELA GODOY,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
3,3,EDUARDO DALA ROSA SOARES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
4,4,PALOMA KIYOMI TAGUCHI LEMOS,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,1797,FERNANDO RIBEIRO SÁ,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6
1798,1798,GABRIELLA GADELHA SARMENTO,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6
1799,1799,MAIELY MARCOLIN,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6
1800,1800,PEDRO HENRIQUE MILHOMENS DA MOTA,SEMANA - TARDE - C.CIRURGICO,VNS,TARDE,sex_1_2,13:00,19:00,06:00,1,NaN,Sexta,6


In [24]:
69+186+517+936+436

2144

In [25]:
for unidade in df['Unidade'].unique():
    separacao = df[df['Unidade'] == unidade]
    for setor in separacao['Setor'].unique():
        print(unidade, '-', setor)

ALPHA - FDS - C.CIRURGICO ROTINA
ALPHA - FDS - PLANTAO
ALPHA - SEMANA - MANHA - C.CIRURGICO
ALPHA - SEMANA - TARDE - C.CIRURGICO
ANALIA - FDS - AVALIACAO PRE ROTINA
ANALIA - FDS - C.CIRURGICO ROTINA
ANALIA - FDS - D1
ANALIA - FDS - PLANTAO
ANALIA - SEMANA - MANHA - AMBULATORIO
ANALIA - SEMANA - MANHA - AV.PRE ANESTESICA
ANALIA - SEMANA - MANHA - C.CIRURGICO
ANALIA - SEMANA - TARDE - AV.PRE ANESTESICA
ANALIA - SEMANA - TARDE - C.CIRURGICO
AVICCENA - FDS - C.CIRURGICO ROTINA
AVICCENA - FDS - D1
AVICCENA - FDS - PLANTAO
AVICCENA - SEMANA - MANHA - C.CIRURGICO
AVICCENA - SEMANA - TARDE - C.CIRURGICO
ITAIM - FDS - AVALIACAO PRE ROTINA
ITAIM - FDS - C.CIRURGICO ROTINA
ITAIM - FDS - D1
ITAIM - FDS - PLANTAO
ITAIM - SEMANA - MANHA - AV.PRE ANESTESICA
ITAIM - SEMANA - MANHA - C.CIRURGICO
ITAIM - SEMANA - TARDE - AV.PRE ANESTESICA
ITAIM - SEMANA - TARDE - C.CIRURGICO
JABAQUARA - FDS - AVALIACAO PRE ROTINA
JABAQUARA - FDS - C.CIRURGICO ROTINA
JABAQUARA - FDS - D1
JABAQUARA - FDS - PLANTAO
JABAQUA

In [26]:
#df['controle'] = ''

In [27]:
df.to_excel("df_escala.xlsx", index=False)

In [28]:
df.head(50)

,Unnamed: 0,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO,SEMANA,controle,dia,semana
0,0,HEDICLEBER PESTANA SEVERINO,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
1,1,KAIÊ FERNANDES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
2,2,LUCCAS VARELA GODOY,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_1_1,07:00,13:00,06:00,1,OK,Sabado,7
3,3,EDUARDO DALA ROSA SOARES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
4,4,PALOMA KIYOMI TAGUCHI LEMOS,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
5,5,THIAGO SILVA PINTO,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_2_1,07:00,13:00,06:00,2,OK,Sabado,7
6,6,HEDICLEBER PESTANA SEVERINO,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_3_1,07:00,13:00,06:00,3,OK,Sabado,7
7,7,LETÍCIA FERNANDES BORGES,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_3_1,07:00,13:00,06:00,3,OK,Sabado,7
8,8,TURÍBIO RODRIGUES FREIRE NETO,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_3_1,07:00,13:00,06:00,3,OK,Sabado,7
9,9,GABRIELA SOARES PIASSI,FDS - C.CIRURGICO ROTINA,ALPHA,DIA,sab_4_1,07:00,13:00,06:00,4,OK,Sabado,7


# Execução

In [29]:
print(len(fd))
print(len(df))

1802
1802


In [30]:
len(df)

1802

In [31]:
H = 382

In [87]:
for i in range(H, len(df)):
    print('contagem', i)
    
    print('individuo:', df['Nome'][i])
    print('Unidade:', df['Unidade'][i])
    print('Setor:', df['Setor'][i])
    print('Dia:', df['dia'][i])
    print('\n')
    
    try:
        if i >= 1:
            if (df['Unidade'][i] == df['Unidade'][i-1]) and (df['Setor'][i] == df['Setor'][i-1]):
                #print('sem mudança de setor=')
                continue
            else:
                print('mudança de setor',"="*50)
                mudar_unidade_setor(i)
        else:
            mudar_unidade_setor(i)
    except:
        print("deu erro nessa porra: -> linha:", i)
    finally:
        sleep(1)
        criar_registro(i)
        df['controle'][i] = "OK"
        H += 1
        sleep(1)

contagem 1781
individuo: VITOR OLIVEIRA ANDRÉ
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Terça


contagem 1782
individuo: BRUNO FERRAZ DE CAMPOS BARONE
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1783
individuo: FABIANA BARBOSA BASTOS
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1784
individuo: MARIA CAROLINA CELIA SILVEIRA
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1785
individuo: PEDRO HENRIQUE MILHOMENS DA MOTA
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1786
individuo: SHEILA ITIROCO MONTE
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1787
individuo: THIAGO FRANCISCO MARTINS
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1788
individuo: VIVIANNA BOVÉRIO GALZERANI
Unidade: VNS
Setor: SEMANA - TARDE - C.CIRURGICO
Dia: Quarta


contagem 1789
individuo: ANDRÉ LUIS XAVIER FRANCO
Unidade: VNS
Setor: SEMANA - TARDE - 

In [59]:
df[df['Nome'] == 'Saboia']

,Unnamed: 0,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO,SEMANA,controle,dia,semana
688,688,Saboia,FDS - C.CIRURGICO ROTINA,JABAQUARA,DIA,sab_2_1,07:00,13:00,06:00,2,NaN,Sabado,7
1230,1230,Saboia,FDS - C.CIRURGICO ROTINA,OSASCO,DIA,sab_3_1,07:00,13:00,06:00,3,NaN,Sabado,7


In [79]:
df['controle'][H] = 'ERRO'
H += 1

In [ ]:
df['dia'].unique()

In [ ]:
for i in range(0, H):
    if i <= H:
        df['controle'][i] = 'OK'
    else:
        df['controle'][i] = 'ok'

In [ ]:
df['Unidade'][df['controle'] != 'OK'].unique()

In [ ]:
df.to_excel("ESCALA_FAZER.xlsx", index=False)

In [ ]:
df.iloc[H]

In [ ]:
H += 1

In [ ]:
fd.iloc[H]

In [ ]:
df['Nome'][H]
df['controle'][i] = "OK"
H += 1

In [ ]:
df['Nome'][df['Nome'] == 'IZABELA CARDOSO GARCIA']

In [ ]:
df['Nome'][df['Nome'] == 'IZABELA CARDOSO GARCIA'] = 'ISABELA CARDOSO GARCIA'

In [ ]:
df['Nome'] = df['Nome'].apply(lambda x: unidecode.unidecode(x.upper().strip()))

# Apagando os modelos ja criados:

In [72]:
df['Setor'].unique()

array(['FDS - C.CIRURGICO ROTINA', 'FDS - PLANTAO',
       'SEMANA - MANHA - C.CIRURGICO', 'SEMANA - TARDE - C.CIRURGICO',
       'FDS - AVALIACAO PRE ROTINA', 'FDS - D1',
       'SEMANA - MANHA - AMBULATORIO',
       'SEMANA - MANHA - AV.PRE ANESTESICA',
       'SEMANA - TARDE - AV.PRE ANESTESICA',
       'SEMANA - MANHA - C.DIAGNOSTICO', 'SEMANA - TARDE - C.DIAGNOSTICO',
       'FDS - MANHA - C.CIRURGICO'], dtype=object)

In [73]:
scrap_unidades

,unidade,momento,plt
0,ADMINISTRACAO,ADMINISTRATIVO,2
1,ADMINISTRACAO,MES ANTERIOR,3
2,ADMINISTRACAO,PERIODO DE DESCANSO - 1,4
3,ADMINISTRACAO,PERIODO DE DESCANSO - 2,5
4,ADMINISTRACAO,PERIODO DE DESCANSO - 3,6
...,...,...,...
194,VNS,SEMANA - TARDE - C.CIRURGICO,208
195,VNS,SEMANA - TARDE - COBERTURA EXTRA,209
196,VNS,SEMANA - NOTURNO - DISTANCIA,210
197,VNS,SEMANA - NOTURNO - PLANTAO,211


In [74]:
fd

,Nome,Setor,Unidade,Periodo,tempo,ENTRADA,SAIDA,DURAÇÃO,SEMANA,controle,dia,semana
0,NATAN SOARES SILVEIRA,SEMANA - MANHA - AV.PRE ANESTESICA,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
1,VANESSA PACIULLO MAROSSI,SEMANA - MANHA - AMBULATORIO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
2,ALEXSANDER TAKASHI HAMADA,SEMANA - MANHA - C.CIRURGICO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
3,ANA JUNG DOCKO,SEMANA - MANHA - C.CIRURGICO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
4,CARLOS EDUARDO CORRÊA GRILO,SEMANA - MANHA - C.CIRURGICO,ANALIA,MANHÃ,seg_1_1,07:00,13:00,06:00,1,ok,Segunda,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1797,MAXVILANO LIMA DE MACEDO,FDS - PLANTAO,ANALIA,NOTURNO,sab_4_2,19:00,07:00,12:00,4,ok,Sabado,7
1798,LEANDRO OLIVEIRA MARTINS CÔRTES,FDS - PLANTAO,ITAIM,TARDE,sab_4_2,13:00,19:00,06:00,4,ok,Sabado,7
1799,LEANDRO OLIVEIRA MARTINS CÔRTES,FDS - PLANTAO,ITAIM,NOTURNO,sab_4_2,19:00,07:00,12:00,4,ok,Sabado,7
1800,THAIS CHAVES PINHEIRO,FDS - PLANTAO,MATERNIDADE,TARDE,sab_4_2,13:00,19:00,06:00,4,ok,Sabado,7


In [77]:
for unidade in fd['Unidade'].unique():
    selecao = fd[fd['Unidade'] == unidade]
    #print(unidade)
    for setor in selecao['Setor'].unique():
        if len(re.findall(r'D[0-9]', setor)) > 0:
            continue
        
        print(unidade, '->', setor)
        #abrindo aba
        browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[1]/div/div[1]/div/div[2]/i[1]').click()
        WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))

        ind = scrap_unidades.loc[(scrap_unidades['unidade'] == unidade) & (scrap_unidades['momento'] == setor)].index.values[0]
        loc = scrap_unidades['plt'][ind]

        #selecionando setores
        browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[1]/div/div[1]/div/div[3]/div/ul/li['+str(loc)+']').click()
        WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
        
        #Verificando se ja tem modelo
        if browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[2]/div[1]/h1[2]/a').text == "Novo Modelo":
            print("não tem modelo")

            #criando modelo zerado
            browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[2]/div[1]/h1[2]/a').click()
            WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
        else:
            print("ja tem modelo")

            #limpando modelo
            browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[2]/div[2]/button[4]').click()
            #deseja continuar
            browser.find_element("xpath", '/html/body/div[4]/div[3]/button[1]').click()
            WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))
        print('\n')
        
print("="*10, "TERMINOU", "="*10)

ANALIA -> SEMANA - MANHA - AV.PRE ANESTESICA
ja tem modelo


ANALIA -> SEMANA - MANHA - AMBULATORIO
ja tem modelo


ANALIA -> SEMANA - MANHA - C.CIRURGICO
ja tem modelo


ANALIA -> SEMANA - TARDE - AV.PRE ANESTESICA
ja tem modelo


ANALIA -> SEMANA - TARDE - C.CIRURGICO
ja tem modelo


ANALIA -> FDS - AVALIACAO PRE ROTINA
ja tem modelo


ANALIA -> FDS - C.CIRURGICO ROTINA
ja tem modelo


ANALIA -> FDS - PLANTAO
ja tem modelo


VILLA -> SEMANA - MANHA - AV.PRE ANESTESICA
ja tem modelo


VILLA -> SEMANA - MANHA - AMBULATORIO
ja tem modelo


VILLA -> SEMANA - MANHA - C.CIRURGICO
ja tem modelo


VILLA -> SEMANA - TARDE - AV.PRE ANESTESICA
ja tem modelo


VILLA -> SEMANA - TARDE - C.CIRURGICO
ja tem modelo


VILLA -> FDS - AVALIACAO PRE ROTINA
ja tem modelo


VILLA -> FDS - C.CIRURGICO ROTINA
ja tem modelo


VILLA -> FDS - PLANTAO
ja tem modelo


AVICCENA -> SEMANA - MANHA - C.CIRURGICO
ja tem modelo


AVICCENA -> SEMANA - TARDE - C.CIRURGICO
ja tem modelo


AVICCENA -> FDS - C.CIRURGICO RO

In [ ]:
for unidade in fd['Unidade'].unique():
    selecao = fd[fd['Unidade'] == unidade]
    print(unidade)
    for setor in selecao['Setor'].unique():
        print('--',setor)
        if len(re.findall(r'D[0-9]', setor)) > 0:
            continue
            print('------------------------------------')
        print("oi")

In [ ]:
fd.drop(21, inplace=True)

In [ ]:
setor

In [ ]:
browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[2]/div[2]/button[4]').click()

In [ ]:
browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/div[2]/div[2]/button[4]').click()

# testes:

In [ ]:
H

In [ ]:
Y = 0

for i in range(Y, len(df)):
    print('contagem', i)

    if i >= 1:
        if (df['Unidade'][i] == df['Unidade'][i-1]) and (df['Setor'][i] == df['Setor'][i-1]):
            print('if 1')
            continue
        else:
            print('else 1')
    else:
        print("else 2")
    
    print('roda================')
    print("\n")
    Y += 1

In [ ]:
H = 0

for i in range(H, len(df)):
    print('contagem', i)
    
    try:
        if i >= 1:
            if (df['Unidade'][i] == df['Unidade'][i-1]) and (df['Setor'][i] == df['Setor'][i-1]):
                print('if 1')
                continue
            else:
                print('else 1')
        else:
            print("else 2")
    except:
        print('deu erro nessa porra')
    finally:
        print('roda================')
        print("\n")
        H += 1

In [ ]:
df = pd.read_excel("ESCALA_FAZER.xlsx")
df

In [ ]:
for unidade in df['Unidade'].unique():
    selecao = df[df['Unidade'] == unidade]
    print(len(selecao), '=', unidade, "="*35)
    #print(unidade)
    for setor in selecao['Setor'].unique():
        print(len(selecao[selecao['Setor'] == setor]), '=', setor)
        
    print('\n')

In [ ]:
fd = pd.DataFrame(columns=['Nome','Setor','Unidade','Periodo','tempo']) #df vazio

count = 0

lista_abas = ['seg','ter','qua','qui','sex']
periodo = "NOITE"

for h in lista_abas:
    tempo = h
    df = pd.read_excel('Escala NOITE SEMANA.xlsx', index_col=periodo, sheet_name=tempo)

    for coluna in df.columns: #para cada coluna
        #print(coluna, '='*20)
        for ind in df.index: #para cada index
            #print(ind,'='*10)
            if pd.isnull(df.loc[ind, coluna]) == True: #verificar se == NA
                continue
            elif (df.loc[ind, coluna] == 'EXTRA') or (df.loc[ind, coluna] == 'Cobertura'):
                continue
            else: #capturar o nome, setor, unidade
                fd.loc[count] = [df.loc[ind, coluna].replace("*",""), ind, coluna, periodo, tempo]

                count += 1
        
    fd['Unidade'] = fd['Unidade'].apply(lambda x: unidecode.unidecode(x.upper()))

fd

In [ ]:
fd['Periodo'].unique()

In [ ]:
df['Periodo'][df['Setor'] == "19-23: HA"] = 'NOITE'
df['Periodo'][df['Setor'] == "19-23: HA"] = 'NOITE'
df['Periodo'][df['Setor'] == "19-23: HA"] = 'NOITE'
df['Periodo'][df['Setor'] == "19-23: HA"] = 'NOITE'
df['Periodo'][df['Setor'] == "19-23: HA"] = 'NOITE'
df['Periodo'][df['Setor'] == "19-23: HA"] = 'NOITE'

In [ ]:
fd['ENTRADA'] = 
fd['SAIDA'] = 
fd['DURAÇÃO'] = 
fd['SEMANA'] = 

In [ ]:
df.head(2)

In [ ]:
df.loc[ind, coluna]

In [ ]:
df[df['Nome'] == "YURI ANDERSON PACHECO DE LIMA"]

In [ ]:
df.to_excel("Escala.xlsx", index=False)